In [1]:
# ! wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream.xml.bz2

In [2]:
!wget https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2

--2022-04-17 07:47:27--  https://dumps.wikimedia.org/zhwiki/20220401/zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215920539 (206M) [application/octet-stream]
Saving to: ‘zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20220401-pag 100%[===================>] 205.92M  4.49MB/s    in 45s     

2022-04-17 07:48:12 (4.62 MB/s) - ‘zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2’ saved [215920539/215920539]



In [3]:
!pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 5.1 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486152 sha256=4e8d3bb1bc4535d80e1c6d23bd5a42b39bfcb56f5c51218748c3db5d2eefc207
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [4]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20220401-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [5]:
wiki_corpus

In [6]:
next(iter(wiki_corpus.get_texts()))[:10]

['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '从某种角度看屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

In [7]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))

10000 articles processed.
20000 articles processed.
30000 articles processed.
32329 articles processed.


In [8]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
train_data = cc.convert(train_data)
train_data = jieba.lcut(train_data)
train_data = [word for word in train_data if word != '']
train_data = ' '.join(train_data)
open('seg.txt', 'w', encoding='utf-8').write(train_data)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.017 seconds.
Prefix dict has been built successfully.


129900316

In [9]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
vector_size = 100
min_count = 1
workers = 8
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [10]:
from gensim.models import word2vec

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)

電腦
('計算機', 0.7770804166793823)
('軟體', 0.7660993933677673)
('硬體', 0.7558894157409668)
('pda', 0.7457395792007446)
('遊戲機', 0.7406581044197083)
('程式', 0.7317909002304077)
('pc', 0.7210907936096191)
('ibm', 0.7112294435501099)
('手機', 0.7037252187728882)
('硬件', 0.7028473019599915)
